# SANS Direct Beam Iteration

This notebook is used for the iterative proceedure of determining the wavelength-dependent direct-beam spectrum, a factor in the normalization term required in SANS data reduction.
For the LoKI detectors this factor is thought to also depend on the layer of straws, or even the individual straw.

In this annotated notebook we are also demoing various ways scipp and scipp's plotting can be used for visualizations and determining and defining masks.

## Definitions

We start by defining functions required for the direct-beam iteration. We suggest to simply run these cells initially, and come back later for a closer look at the code, if desired.

In [ ]:
import scipp as sc
import numpy as np
import sans
import contrib

Helper function for running a 1D Mantid fit on all slices of a 2D array:

In [ ]:
def fit_gauss_coil(data, flat_background):
    import functools
    import gauss_coil_fit1
    model = "polyGaussCoil"
    params = f"I0=60.0,Rg=50.0,Mw_by_Mn=1.02,Background={flat_background.value}"
    ties = "Rg=50.0,Mw_by_Mn=1.02"
    def fit_layer(i):
        return sc.neutron.fit(data['layer', i],
                              mantid_args={'Function':f'name={model},{params}', 'Ties':ties})
    params, diff = zip(*map(fit_layer, range(data.coords['layer'].shape[0])))
    concat = functools.partial(sc.concatenate, dim='layer')
    return functools.reduce(concat, params), functools.reduce(concat, diff)

In [ ]:
def normalize_and_subtract(sample, background):
    sample_norm = sample['data'] / sample['norm']
    background_norm = background['data'] / background['norm']
    return sample_norm - background_norm

def q_range_mask(wavelength):
    # Ratio of reduced_band / reduced is averaged (integrated) over wavelength-dependent interval
    inv_w = sc.reciprocal(wavelength)
    dim = inv_w.dims[0]
    d_inv_w = sc.concatenate(inv_w[dim,1:], inv_w[dim,:-1], 'bound') - inv_w[dim,-1]
    q_range = qlongW + d_inv_w * (qshortW - qlongW) / (inv_w[dim,0] - inv_w[dim,-1])
    qmin = q_range['bound',0]
    qmax = q_range['bound',1]
    return sc.greater(qmin, q_bins['Q',:-1]) | sc.less(qmax, q_bins['Q',1:])

def interpolate_cubic_spline(data, x, dim):
    import functools
    from scipy import interpolate
    def interpolate_layer(i):
        tck = interpolate.splrep(
            x=contrib.midpoints(data.coords[dim], dim).values,
            y=data['layer',i].values,
            w=1.0/np.sqrt(data['layer',i].variances))
        # TODO uncertainties
        return sc.Variable(dims=[dim], values=interpolate.splev(x.values, tck))
    y = map(interpolate_layer, range(data.coords['layer'].shape[0]))
    concat = functools.partial(sc.concatenate, dim='layer')
    return sc.DataArray(data=functools.reduce(concat, y), coords={dim:x})

def to_q_by_wavelength(data, transmission, direct_beam, direct_beam_transmission, masks, wavelength_bands):
    wav = sans.to_wavelength(data=data,
                             transmission=transmission,
                             direct_beam=direct_beam,
                             direct_beam_transmission=direct_beam_transmission,
                             masks=masks,
                             wavelength_bins=wavelength_bins)
    return sans.reduce_by_wavelength(wav, q_bins, groupby='layer', wavelength_bands=wavelength_bands)

def direct_beam_iteration(direct_beam, layers, masks, wavelength_bands, flat_background=0.25*sc.units.one):
    print('start iteration')
    direct_beam = direct_beam.copy()
    direct_beam_by_pixel = sc.choose(layers, choices=direct_beam, dim='layer')
    sample_q_lambda = to_q_by_wavelength(data=sample,
                                         transmission=sample_trans,
                                         direct_beam=direct_beam_by_pixel,
                                         direct_beam_transmission=background_trans, # note: background_trans
                                         masks=masks,
                                         wavelength_bands=wavelength_bands)
    background_q_lambda = to_q_by_wavelength(data=background,
                                             transmission=background_trans,
                                             direct_beam=direct_beam_by_pixel,
                                             direct_beam_transmission=background_trans, # note: same as transmission
                                             masks=masks,
                                             wavelength_bands=wavelength_bands)

    # reduced by wavelength band
    reduced_by_wavelength = normalize_and_subtract(sample_q_lambda, background_q_lambda)
    
    # sum wavelength bands to reduce for full range
    sample_q1d = sc.sum(sample_q_lambda, 'wavelength')
    background_q1d = sc.sum(background_q_lambda, 'wavelength')
    reduced = normalize_and_subtract(sample_q1d, background_q1d)
    
    # store I(Q) before scaling for inspecting intermediate terms
    direct_beam.attrs['I(Q)'] = sc.Variable(value=reduced)
    direct_beam.attrs['I(Q,lambda)'] = sc.Variable(value=reduced_by_wavelength)
    
    params, diff = fit_gauss_coil(contrib.select_bins(reduced, 'Q', qlongW['bound', 0], qshortW['bound', 1]),
                                  flat_background=flat_background)
    scale = I0_expected / params['parameter', 0].data
    reduced_by_wavelength *= scale
    reduced *= scale
    direct_beam /= scale
    ratio = (reduced_by_wavelength - flat_background) / (reduced - flat_background)
    ratio.masks['Q-range-mask'] = q_range_mask(wavelength_bands)
    norm = 1.0*sc.units.one + damp*(sc.mean(ratio, 'Q') - 1.0*sc.units.one)
    splined = interpolate_cubic_spline(norm, direct_beam.coords['wavelength'], 'wavelength')
    direct_beam *= splined
    
    # store some information for inspecting intermediate terms
    direct_beam.attrs['fit'] = sc.Variable(value=diff)
    direct_beam.attrs['knots'] = sc.Variable(value=norm)
    direct_beam.attrs['spline'] = sc.Variable(value=splined)
    
    return direct_beam

Set filenames to use for the data reduction and iteration:

In [ ]:
import os

try:
    import dataconfig # run make_config.py to create this
except:
    print("ERROR: dataconfig.py not find please run `make_config.py`\n")
    !python make_config.py -h # change to `-f path` or run in terminal

path = os.path.join(dataconfig.data_root, 'ess', 'loki', 'tube-calibration')
direct_beam_file = 'DirectBeam_20feb_full_v3.dat' # same as DirectBeam_28Apr2020.dat
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
sample_run_number = 49338
sample_transmission_run_number = 49339
background_run_number = 49334
background_transmission_run_number = 49335

def load_larmor(run_number):
    return sc.neutron.load(filename=f'{path}/LARMOR000{run_number}.nxs')

def load_rkh(filename):
    return sc.neutron.load(
           filename=filename,
           mantid_alg='LoadRKH',
           mantid_args={'FirstColumnValue':'Wavelength'})

In [ ]:
def load_mask(idf, mask):
    return sc.neutron.load(
           filename=idf,
           mantid_alg='LoadMask',
           mantid_args={'InputFile':mask})

Load files:

In [ ]:
%%time
sample_trans = load_larmor(sample_transmission_run_number)
sample = load_larmor(sample_run_number)
background_trans = load_larmor(background_transmission_run_number)
background = load_larmor(background_run_number)

Mantid mask files could be loaded as follows:

In [ ]:
idf_filename = f'{path}/LARMOR_Definition.xml'
mask_filename = f'{path}/Mask_full.xml'
mask = sc.neutron.load(filename=idf_filename, mantid_alg='LoadMask', mantid_args={'InputFile':mask_filename})
sc.neutron.instrument_view(mask, pixel_size=0.01)

## Visualizations and mask setup

A central requirement during commissioning and operation are visualizations for raw data to find, e.g., broken detectors or other artifacts that need to be masked.
The data array with our raw data looks as follows:

In [ ]:
sample

### Straw plot against real X

In [ ]:
vals = sample.values

In [ ]:
from loki import LoKI
loki = LoKI()
from scipp.plot import plot
spectrum_counts = sc.sum(sample, 'tof') # sum is optional, could also keep TOF
spectrum_counts.coords['pixel'] = sc.geometry.x(sample.coords['position'])
pixel_counts = loki.to_logical_dims(spectrum_counts) # reshape
plot(pixel_counts, norm='log', axes={'y':'tube', 'x':'pixel'})

### Bad straws

The LoKI detectors are tubes containing 7 straws each, and there are multiple layers of tubes.
This makes finding, e.g., broken straws in the instrument view difficult and tedious.
With scipp we can reshape our data to match this logical layer and sum, e.g., over time-of-flight and pixels within straws.
This yields:

In [ ]:
from scipp.plot import plot
from loki import LoKI
loki = LoKI()
spectrum_counts = sc.sum(sample, 'tof') # sum is optional, could also keep TOF
pixel_counts = loki.to_logical_dims(spectrum_counts) # reshape
straw_counts = sc.sum(pixel_counts, 'pixel')
plot(straw_counts, norm='log', vmin=1, vmax=1e7)

In this case we observe 4 straws with 0 counts as well as 4 straws with very low counts.
We can define a mask for these using a small LoKI-specific helper:

In [ ]:
bad_straws = sc.Variable(dims=['spectrum'], shape=sample.coords['spectrum'].shape, dtype=sc.dtype.bool)
mask = sc.Variable(value=True)
bad_straws[loki.straw(tube=0, straw=6)] |= mask
bad_straws[loki.straw(tube=1, straw=2)] |= mask
bad_straws[loki.straw(tube=5, straw=1)] |= mask
for straw in 1,2,3,4,5:
    bad_straws[loki.straw(tube=12, straw=straw)] |= mask
sample.masks['bad-straws'] = bad_straws

We may also want to inspect and compare individual straws.
This just serves an example here with no actual artifacts visible in this case:

In [ ]:
plot({f'tube-{tube}-straw{straw}':pixel_counts['tube', tube]['straw', straw]
      for tube, straw in [(1,3),(3,5),(4,0),(11,3)]}, norm='log', marker='.')

### Bad pixels

Due to readout electronics issues there are also ranges of pixels within certain straws in the high-intensity are near the beam.
We use the array of pixel counts computed above, but do not sum pixels with a straw this time.
This results in a 3D volume and we can use the sliders in the slice viewer to inspect individual straws:

In [ ]:
plot(pixel_counts, vmax=100, axes={'x':'pixel', 'y':'tube'})

The dark blue areas near the center are the areas that require masking.
Instead of listing those pixels directly we would like to mask pixels with zero counts.
Towards the tube ends there are "real" zeros though (area with actual low count rates), so we use a combined condition with distance from the beam center:

In [ ]:
pos = sc.neutron.position(sample)
x = sc.geometry.x(pos)
y = sc.geometry.y(pos)
counts = spectrum_counts.data
counts.variances = None # TODO support comparison with variances
sample.masks['electronics-error'] = sc.less(sc.abs(x), 0.2 * sc.units.m) \
                                  & sc.less(sc.abs(y), 0.03 * sc.units.m) \
                                  & sc.equal(counts, 0.0 * sc.units.counts)
print(f"Masking {sc.sum(sample.masks['electronics-error'], 'spectrum').value} bad pixels due to electronics error.")

### Beam stop

We use a simple geometric condition to mask the beam stop.
This will likely require refinement in the future, e.g., taking into account also `z`:

In [ ]:
# Note that this needs more tuning and masks too much. Better do this after moving detectors?
sample.masks['beam-stop'] = sc.less(sc.abs(x), 0.03 * sc.units.m) \
                          & sc.less(y, 0.028 * sc.units.m) \
                          & sc.greater(y, -0.016 * sc.units.m)

### Tube ends

The ends of the LoKI tubes are ouside the beam area in the tests at LARMOR and need to be masked:

In [ ]:
sample.masks['tube-ends'] = sc.greater(x, 0.36 * sc.units.m) \
                          | sc.less(x, -0.36 * sc.units.m)

### Overview

We can recompute the pixel counts to see the various masks we have defined:

In [ ]:
pixel_counts = loki.to_logical_dims(sc.sum(sample, 'tof'))
plot(pixel_counts, vmax=1000, axes={'x':'pixel', 'y':'tube'})

### Instrument view

Standard instrument view showing all pixels.
Not that masks can be hidden or shown in different colors using the widgets:

In [ ]:
loki.instrument_view(sample, norm='log')

Alternatively, we can use `groupby` to group pixels based on some criterion and then use the `copy` method to extract the desired group.

#### Show single layer

Note: This cell can be skipped.

Detectors with multiple layers are not adequately visualized in a normal instrument view since internal pixels are invisible.
We can selected layers of choice and show them individually:

In [ ]:
from loki import LoKI
loki = LoKI()
sample.coords['layer'] = loki.layers()
sc.neutron.instrument_view(sc.groupby(sample, 'layer').copy(group=1), norm='log')

In [ ]:
del sample.coords['layer']

#### Show pixels with low number of counts

Note: This cell can be skipped.

Using the same mechanism we can create nearly arbitrary other visualizations of the instrument.
For example, we may want to inspect all pixels with low counts rates, e.g., to find issues with detectors.
In this case mask all pixels with less than 100 counts and can check whether we indeed masked all relevant features:

In [ ]:
sample.coords['low-counts'] = sc.less(counts, 100.0*sc.units.counts)
sc.neutron.instrument_view(sc.sum(sc.groupby(sample, 'low-counts').copy(group=1), 'tof'))

In [ ]:
del sample.coords['low-counts']

### Promt pulse mask

We use a comparison of time-of-flight to define a promt-pulse mask:

In [ ]:
tof = sample.coords['tof']
sample.masks['prompt-pulse'] = sc.less(tof['tof',1:], 1500.0 * sc.units.us) | \
                              (sc.greater(tof['tof',:-1], 17500.0 * sc.units.us) & \
                               sc.less(tof['tof',1:], 19000.0 * sc.units.us))

## Reduction and direct-beam iteration

We are now ready to proceed with the actual direct-beam iteration.

Move sample and detectors to their actual positions:

In [ ]:
%%time
dtype = sample.coords['position'].dtype
sample_pos_offset = sc.Variable(value=[0.0, 0.0, 0.30530], unit=sc.units.m, dtype=dtype)
bench_pos_offset = sc.Variable(value=[0.0, 0.001, 0.0], unit=sc.units.m, dtype=dtype)
for item in [sample, sample_trans, background, background_trans]:
    item.coords['sample-position'] += sample_pos_offset
    item.coords['position'] += bench_pos_offset

Define desired $Q$ and $\lambda$ binning:

In [ ]:
q_bins = sc.Variable(
    dims=['Q'],
    unit=sc.units.one/sc.units.angstrom,
    values=np.geomspace(0.007999, 0.6, num=55))
wavelength_bins = sc.Variable(
    dims=['wavelength'],
    unit=sc.units.angstrom,
    values=np.geomspace(1.0, 12.9, num=110))
wavelength_bands = sc.Variable(
    dims=['wavelength'],
    unit=sc.units.angstrom,
    values=np.geomspace(1.0,12.0,num=10))

Parameters required for iteration:

In [ ]:
I0_expected = 55.77 * sc.units.one

In [ ]:
qlongW = sc.Variable(dims=['bound'], unit=q_bins.unit, values=[0.008, 0.05])
qshortW = sc.Variable(dims=['bound'], unit=q_bins.unit, values=[0.1, 0.22])
damp = 1.0*sc.units.one

Load direct beam "starting guess" and use it for all layers:

In [ ]:
%%time
from loki import LoKI
loki = LoKI()
layers = loki.layers()
n_layer = sc.max(layers).value + 1
direct_beam = load_rkh(filename=f'{path}/{direct_beam_file}')
# Use same starting value for all layers
direct_beam = sc.Variable(dims=['layer'], values=np.ones(n_layer)) * direct_beam
direct_beam.coords['layer'] = sc.Variable(dims=['layer'], dtype=sc.dtype.int32, values=np.arange(n_layer))

Run the actual iteration.
Internally this is essentially computing 3D $I(\text{layer}, \lambda\text{-band},Q)$, i.e., $I(Q)$ as produced by a normal SANS data reduction, but with 2 extra dimensions.
This is then used to compute corrections to the current direct-beam:

In [ ]:
%%time
direct_beams = [direct_beam]
for i in range(4):
    direct_beams.append(direct_beam_iteration(direct_beams[-1],
                                              layers=layers,
                                              masks=sample.masks,
                                              wavelength_bands=wavelength_bands))
direct_beams = sc.Dataset({f'iteration-{i}':item for i, item in enumerate(direct_beams)})

The direct-beam iteration is a process that requires experimentation and inspection of intermediate terms and results.
We can plot the direct beam at individual iterations, or for individual layers.
Intermediate terms such as fit results or $I(\text{layer}, \lambda\text{-band},Q)$ are stored as attributes:

In [ ]:
direct_beams[f'iteration-1'].attrs['I(Q,lambda)'].value

In [ ]:
direct_beams[f'iteration-1']['layer',1]

Examples of results and intermediate terms are plotted below:

In [ ]:
from scipp.plot import plot
plot(direct_beams['layer',1], marker='.')
plot(sc.collapse(direct_beams['iteration-4'], keep='wavelength'), marker='.')

In [ ]:
plot(sc.collapse(direct_beams['iteration-4'].attrs['knots'].value['wavelength',:130], keep='wavelength'))

In [ ]:
plot(sc.collapse(direct_beams['iteration-4'].attrs['spline'].value['wavelength',:130], keep='wavelength'))

In [ ]:
iteration = 1
layer = 1

def wav_band_label(coord, i):
    left = coord['wavelength', i].value
    right = coord['wavelength', i+1].value
    return f'{round(left,2)}-{round(right,2)} {coord.unit}'

db = direct_beams[f'iteration-{iteration}']
I = db.attrs['I(Q,lambda)'].value['layer',layer]
items = {wav_band_label(I.coords['wavelength'], i):I['wavelength', i]
         for i in range(len(I.coords['wavelength'].values)-1)}
items['full range'] = db.attrs['I(Q)'].value['layer',layer]
items['fit'] = db.attrs['fit'].value['layer',layer]['calculated']
plot(items, norm='log')

In [ ]:
layer_index = 1
for i in 1,2,4:
    from IPython.display import display
    display(plot(direct_beams[f'iteration-{i}'].attrs['fit'].value['layer', layer_index]))